In [0]:
df_crm_sales_details = spark.table(
    "bronze.crm_sales_details"
)

In [0]:
df_crm_sales_details.display()

- Check the Schema of the table

In [0]:
df_crm_sales_details.printSchema()

- Rename Column Names

In [0]:
print(df_crm_sales_details.columns)

In [0]:
mapping_column_names = {
    'sls_ord_num': 'order_number' ,
    'sls_prd_key': 'product_key', 
    'sls_cust_id': 'customer_id', 
    'sls_order_dt': 'order_date', 
    'sls_ship_dt': 'ship_date', 
    'sls_due_dt': 'due_date', 
    'sls_sales': 'total_sales', 
    'sls_quantity': 'quantity', 
    'sls_price': 'price'
}

In [0]:
# create a function that renames the column name

def rename_column(dataframe, old_column, new_column):
    return dataframe.withColumnRenamed(
        old_column, new_column
    )

In [0]:
# update the column names

for old_col, new_col in mapping_column_names.items():
    df_crm_sales_details = rename_column(dataframe=df_crm_sales_details, old_column=old_col, new_column=new_col)

In [0]:
print(df_crm_sales_details.columns)

In [0]:
df_crm_sales_details.display()

- Check for Nulls


In [0]:
from pyspark.sql.functions import col,isnan, when, count
df_crm_sales_details.select([count(when(col(c).isNull(), c)).alias(c) for c in df_crm_sales_details.columns]
   ).display()


- Handle Nulls in sales column

In [0]:
%sql
SELECT * 
FROM bronze.crm_sales_details
WHERE sls_sales IS NULL

In [0]:
# replace nulls in sales column , using price * quantity
from pyspark.sql.functions import col, when

df_crm_sales_details = df_crm_sales_details.withColumn(
    'total_sales',
    when(
        col("total_sales").isNull(),
        col("quantity") * col("price")
    )
    .otherwise(col("total_sales"))
)


In [0]:
from pyspark.sql.functions import col,isnan, when, count
df_crm_sales_details.select([count(when(col(c).isNull(), c)).alias(c) for c in df_crm_sales_details.columns]
   ).display()


* handle missing values in price column

In [0]:
%sql
SELECT * 
FROM bronze.crm_sales_details
WHERE sls_price IS NULL

In [0]:
 # replace null values in price column with total_sales / quantity

from pyspark.sql.functions import col, when

df_crm_sales_details = df_crm_sales_details.withColumn(
    'price',
    when(
        col("price").isNull(), col("total_sales") / col("quantity")
    )
    .otherwise(col("price"))
)


In [0]:
from pyspark.sql.functions import col,isnan, when, count
df_crm_sales_details.select([count(when(col(c).isNull(), c)).alias(c) for c in df_crm_sales_details.columns]
   ).display()


In [0]:
df_crm_sales_details.display()

- Trim Columns

In [0]:
# remove extra spaces using trim function
from pyspark.sql.functions import trim, col

for idx, column in enumerate(df_crm_sales_details.columns):
    if df_crm_sales_details.dtypes[idx][1] == 'string':
        df_crm_sales_details = df_crm_sales_details.withColumn(
            column, trim(col(column))
        )

In [0]:
df_crm_sales_details.display()

- Fix All Date Columns, from yyyymmdd, to yyyy-mm-dd

In [0]:
df_crm_sales_details.select("order_date", "due_date", "ship_date").display()

In [0]:
# filter only date rows with length = 8
from pyspark.sql.functions import col, length


for column in ['order_date', 'ship_date', 'due_date']:
    df_crm_sales_details = (
    df_crm_sales_details
    .withColumn(column, col(column).cast("string"))
    .filter((length(col(column)) == 8) & (~col(column).isin("0", "00000000")))   
)

In [0]:
from pyspark.sql.functions import col, when, to_date

for c in ["order_date","ship_date","due_date"]:
    df_crm_sales_details = df_crm_sales_details.withColumn(
        c,
        when(col(c).isin(0, "0"), None).otherwise(col(c))
    ).withColumn(
        c,
        to_date(col(c).cast("string"), "yyyyMMdd")
    )


In [0]:
# check the results
df_crm_sales_details.select('order_date', 'ship_date', 'due_date').display()

In [0]:
# check the schema 

df_crm_sales_details.select('order_date', 'ship_date', 'due_date').printSchema()

In [0]:
df_crm_sales_details.display()

- Extract as silver table

In [0]:

df_crm_sales_details.write.format("delta").mode("overwrite").saveAsTable("silver.crm_sales_details_info")
